# Calculating Bond Payment Dates and Amounts

## Importing Libraries, Modules, And Functions

As in Chapter One of the volume, modules that are included in the standard Python library are imported. When necessary, other modules or libraries are installed before they are imported. If necessary, The new module <font color='green'>holidays</font> is installed and then imported.  

```
import sys
import requests
from datetime import datetime, date
from types import ModuleType
import calendar

try:
    from IPython.display import Markdown, display
except:
    !pip -q install IPython
    from IPython.display import Markdown, display

try:
    import numpy as np
except:
    !pip -q install numpy
    import numpy as np

try:
    from dateutil.relativedelta import relativedelta
except:
    !pip install python-dateutil
    from dateutil.relativedelta import relativedelta
    from datetutil.easter import easter

try:
    import holidays
except:
    !pip -q install holidays
    import holidays
```

In [1]:
# Import OS to interact with local computer operating system
import sys
import requests
from types import ModuleType
# Import the datetime and date classes from the datetime module, and  for working with dates.
from datetime import datetime, date
#Import markdown and display
from IPython.display import Markdown as md, display
# Last calendar day of the month and day of the week for first day
import calendar

# Ipython not builtin, install if  necessary with quiet flag (-q)
try:
    from IPython.display import Markdown, display
except:
    !pip -q install IPython
    from IPython.display import Markdown, display

# Pandas not builtin, install if  necessary with quiet flag (-q)
try:
    import pandas as pd
except:
    !pip -q install pandas
    import pandas as pd

# NumPy not builtin, install if  necessary with quiet flag (-q)
try:
    import numpy as np
except:
    !pip -q install numpy
    import numpy as np

# dateutilnot builtin, install if  necessary with quiet flag (-q)
try:
    from dateutil.relativedelta import relativedelta
except:
    !pip -q install python-dateutil
    from dateutil.relativedelta import relativedelta
    from dateutil.easter import easter

## Adding A Custom Module And Importing Functions

Similar to Chapter One, this chapter accesses the custom module *basic_concepts_fixed_income* from Dropbox. Three functions are imported from this module: <font color='green'>validate_date</font> and <font color='green'>schedule_pay_datese</font> (developed in the previous chapter's notebook, *Calculating Accrued Interest*), and <font color='green'>adjust_bond_pay_dates</font> (from this chapter's notebook, *Accounting For Non-Settlement Days: Holidays, Weekends, And Good Friday*).

The practice of developing and adding functions to this custom module for use in subsequent chapters is a recurring feature of the volume, *Basic Concepts Of Fixed Income*.

```py
from basic_concepts_fixed_income import (validate_date,
                                         scheduled_pay_dates,
                                         adjust_bond_pay_dates)
```



In [2]:
import requests, sys
from types import ModuleType
# Define the URL of the Python module to be downloaded from Dropbox.
# The 'dl=1' parameter in the URL forces a direct download of the file content.
url= 'https://www.dropbox.com/scl/fi/4y5hjxlfphh1ngvbgo77q/\
module_-basic_concepts_fixed_income.py?rlkey=6oxi7mgka42veaat79hcv8boz&st=87sztshr&dl=1'
module_name='basic_concepts_fixed_income'
# Send an HTTP GET request to the URL and store the server's response.
try:
  response=requests.get(url)
  # Raise an exception for bad status codes (like 404 Not Found)
  response.raise_for_status()
  module= ModuleType(module_name)
  #Code contained in response.text executed
  exec(response.text, module.__dict__)
  # Module added to sys
  sys.modules[module_name]=module
except requests.exceptions.RequestException as e:
    print(f"❌ Error: Could not fetch module from URL. {e}")
except Exception as e:
    print(f"❌ Error: Failed to execute or import the module. {e}")

# Now that 'basic_concepts_fixed_income' exists in the notebook, import the specific functions
from basic_concepts_fixed_income import (validate_date,
                                         scheduled_pay_dates,
                                         adjust_bond_pay_dates)

# New Section

## The <font color='green'>bond_pay_data</font> function calculates actual bond payment dates and amounts.

 The <font color='green'>bond_pay_data()</font> function orchestrates bond payment calculations using three key helper functions:

1. <font color='green'>validate_date</font>: Confirms the validity of the maturity and settlement dates.  
2. <font color='green'>scheduled_pay_dates</font>: Generates all the bond's scheduled payment dates.  
3. <font color='green'>adjust_bond_pay_dates</font>: Converts the scheduled payment dates to the final settlement dates, as necessary.

Required Arguments:

* <font color='green'>maturity</font>: The bond's maturity date; must be a `date` or `datetime` object.  
* <font color='green'>coupon</font>: The annual coupon amount (for a par value of 100); must be a non-negative floating-point number.

Optional Arguments:

* <font color='green'>settlement</font>: The settlement date, which defaults to the current day; must be a `date` or `datetime` object.  
* <font color='green'>**freq</font>: The annual payment frequency; defaults to semi-annual (2). This must be an integer value from the set {1, 2, 4, 12}, corresponding to annual, semi-annual, quarterly, or monthly payments, respectively.

Function Purpose and Return:

This function determines both the exact dates and the corresponding amounts of a bond's payments. This information is critical for calculating the bond's present value and, as explored in the following chapter, for deriving the zero prices or the term structure of interest rates implied by current bond prices.

The function returns two NumPy arrays: one for the payment dates and one for the payment amounts (<font color='green'>pay_dates</font>, <font color='green'>pay</font>).Payment Calculation Logic:

* **If the coupon is zero:** The function immediately returns a NumPy array containing only the adjusted maturity date and a NumPy array with the par value of 100\.

```py
   adjust_maturity=adjust_bond_pay_dates(maturity)
   return np.array([adjust_maturity]),np.array([100.0])

```

* **If the coupon is positive:**  
  1. A NumPy array <font color='green'>pay</font> is initialized with a length matching the number of dates. Each element is set to the **annual coupon divided by the frequency**..  
  2. The last element of the `pay` array (the payment on the maturity date) is increased by the **par value of 100**.

```py
pay = np.full(len(dates), coupon / freq)
pay[-1] += 100
```


```py
pay_dates=np.array([adjust_bond_pay_dates[scheduled] for scheduled in scheduled_dates])

```

In [3]:
def bond_pay_data(maturity,coupon,settlement=None,freq=2):
  '''
  Function calculates payment Dates And Amounts.
  maturity is a datetime object and coupon is a real number.
  Required arguments are maturity and annual coupon.
  If provided, the value of settlement is a datetime object;
  otherwise defaults to date.today()
  freq defaults to semi-annual but accepts freq equal
  to 1 for annual, 2 for semi-annal, 4 for quarterly, and 12 for monthly.
  The function assumes a par value of 100.
  Returns Numpy arrays of dates and amounts.

   Raises:
      TypeError: If maturity or settlement are not datetime objects.
      ValueError: If inputs are not logically valid (e.g., negative coupon,
                  maturity before settlement).
  '''
  from datetime import datetime,date
  from dateutil.relativedelta import relativedelta
  import pandas as pd
  import numpy as np
  from IPython.display import display, Markdown as md

  #Validate the data- maturity, coupon, settlement, freq
  #maturity
  maturity=validate_date(maturity)

  #coupon
  try:
      coupon = float(coupon)
      if coupon < 0:
          raise ValueError("coupon rate cannot be negative.")
  except (ValueError, TypeError):
      raise ValueError("coupon must be a valid number.")

  #settlement
  if settlement is None:
      settlement = date.today()
  else:
      settlement=validate_date(settlement)

  #freq
  if int(freq) not in [1,2,4,12]:
      display(md(f"### ⚠️  your assigned freq {freq} it must be (1, 2, 4, or 12)\
      \n  ###    semi-annual assumed (2)."))
      freq=int(2)

  if coupon==0:
    #Adjust maturity for non-settlement day and return date and face value
    adjust_maturity=adjust_bond_pay_dates(maturity)
    return np.array([adjust_maturity]),np.array([100.0])

  #get scheduled payment dates from helper function scheduled_pay_dates
  scheduled_dates=scheduled_pay_dates(maturity,settlement,freq)

  #NumPy array of pay_dates: list comprehension with adjust_bond_pay_dates
  pay_dates=np.array([adjust_bond_pay_dates(scheduled) for scheduled in scheduled_dates])

  #calculate payments
  #coupon divided by freq at each date
  pay=np.full(len(pay_dates),coupon/freq)

  #Add principal payment as last cash payment
  pay[-1]+=100

  return pay_dates,pay

## Bond Payment Dates
The function is demonstrated with the February 29$^{th}$ 2028 semi-annual 4.0 Coupon Bond.
The diagram shows the correct pattern of pay dates and payments of the bond.
 February 29$^{th}$ 2028 4.0 Coupon Bond Payment Dates And Amounts
<img src='https://docs.google.com/drawings/d/e/2PACX-1vSaXeujj_5-3TMLbGrVHdZ-RkxmRv4TTkPH5aQ-JCxDsObw5Mj6XyyayVsBIEvAM42rNakP-Zmrlmq_/pub?w=960&h=540'>





In [4]:
maturity=date(2028,2,29)
settlement=date(2025,1,21)
coupon=4
bond_pay_data(maturity,coupon,settlement=settlement)

(array([datetime.date(2025, 2, 28), datetime.date(2025, 9, 2),
        datetime.date(2026, 3, 2), datetime.date(2026, 8, 31),
        datetime.date(2027, 3, 1), datetime.date(2027, 8, 31),
        datetime.date(2028, 2, 29)], dtype=object),
 array([  2.,   2.,   2.,   2.,   2.,   2., 102.]))

## <span style="text-align: left; color:green; font-family: 'Franklin Gothic Medium', sans-serif; margin-top: 1.0em; margin-bottom: 0em; font-style: italic;">Chapter Exercise</span>
<span style="text-align: left; color:green; font-family: 'Franklin Gothic Medium', sans-serif; margin-top: 0; margin-bottom: 0.5em; font-style: italic;"><big>Calculate The Payment Dates And Amounts For Five Bonds</big></font></span>




|Maturity|Coupon|
|-------|-------|
|&nbsp;&nbsp;&nbsp;October 9$^{th}$ 2028|&nbsp;&nbsp;&nbsp;1.50|
|&nbsp;&nbsp;&nbsp;December 31$^{st}$ 2028|&nbsp;&nbsp;&nbsp;5.00|
|&nbsp;&nbsp;&nbsp;November 28$^{th}$ 2030|&nbsp;&nbsp;&nbsp;3.50|
|&nbsp;&nbsp;&nbsp;October 10$^{th}$ 2027|&nbsp;&nbsp;&nbsp;2.50|
|&nbsp;&nbsp;&nbsp;February 18$^{th}$ 2029|&nbsp;&nbsp;&nbsp; 4.00|




 see [Chapter Three Hints: Calculate Bond Pay Data](https://patrickjhess.github.io/Hints-Results/Chapter_Three_Hints.html#chapter-three-exercise), and check the [expected results here](https://patrickjhess.github.io/Hints-Results/Chapter_Three_Results.html#chapter-three-exercise).



## adjust_bond__pay_date helper function

**Purpose:**

Converts scheduled payment dates to actual payment dates by checking for settlement dates.

**Behavior:**

* **Input Validation:** Ensures the integrity of scheduled payment dates using the `validate_date` function.  
* **Error Handling:** Raises an exception if the scheduled date is not a date or datetime object.  
* **Calculations:** Determines the settlement day using the `holiday` library, the `easter` function from `relativedelt.easter`, and the `weekday()` method of the `datetime` object.  
* **Output:** Returns the actual payment date, adjusting the scheduled date to a settlement day if necessary.